# ORGANICS

Анализ данных файла "ORGANICS.csv" 

In [1]:
import numpy as np
import pandas as pd
import pylab as pl
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
import seaborn as sns

data = pd.read_csv('/Users/giniatullinaalsuzamilevna/Desktop/Data/DATA-ANALYSYS_2-master/Dataset/ORGANICS.csv',encoding="cp1251")

Информация о файлe "ORGANICS.csv":

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22223 entries, 0 to 22222
Data columns (total 13 columns):
ID                 22223 non-null int64
DemAffl            21138 non-null float64
DemAge             20715 non-null float64
DemCluster         21549 non-null float64
DemClusterGroup    21549 non-null object
DemGender          19711 non-null object
DemReg             21758 non-null object
DemTVReg           21758 non-null object
PromClass          22223 non-null object
PromSpend          22223 non-null float64
PromTime           21942 non-null float64
TargetBuy          22223 non-null int64
TargetAmt          22223 non-null int64
dtypes: float64(5), int64(3), object(5)
memory usage: 2.2+ MB


Так как имеем столбцы, которые не имеют отношение к анализу (ID, TargetAmt), то сделаем новый dataframe без них:

In [3]:
new_data = data[['DemAffl', 'DemAge', 'DemCluster', 'DemClusterGroup', 'DemGender', 'DemReg', 'DemTVReg', 'PromClass', 'PromSpend', 'PromTime', 'TargetBuy']]

Информация об анализируемом dataframe:

In [4]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22223 entries, 0 to 22222
Data columns (total 11 columns):
DemAffl            21138 non-null float64
DemAge             20715 non-null float64
DemCluster         21549 non-null float64
DemClusterGroup    21549 non-null object
DemGender          19711 non-null object
DemReg             21758 non-null object
DemTVReg           21758 non-null object
PromClass          22223 non-null object
PromSpend          22223 non-null float64
PromTime           21942 non-null float64
TargetBuy          22223 non-null int64
dtypes: float64(5), int64(1), object(5)
memory usage: 1.9+ MB


In [5]:
print(new_data.shape)

(22223, 11)


Статистика по значениям признаков:

In [6]:
new_data.describe()

,DemAffl,DemAge,DemCluster,PromSpend,PromTime,TargetBuy
count,21138.000000,20715.000000,21549.000000,22223.000000,21942.000000,22223.000000
mean,8.711893,53.797152,27.193652,4420.590041,6.564670,0.247716
std,3.421125,13.206048,15.751547,7559.047522,4.657113,0.431696
min,0.000000,18.000000,1.000000,0.010000,0.000000,0.000000
25%,6.000000,44.000000,14.000000,0.010000,4.000000,0.000000
50%,8.000000,54.000000,27.000000,2000.000000,5.000000,0.000000
75%,11.000000,64.000000,38.000000,6000.000000,8.000000,0.000000
max,34.000000,79.000000,55.000000,296313.850000,39.000000,1.000000


Проверка корреляции между признаками:

In [7]:
data_corr = new_data.corr()

In [8]:
data_corr

,DemAffl,DemAge,DemCluster,PromSpend,PromTime,TargetBuy
DemAffl,1.000000,-0.137767,0.028571,-0.019621,-0.029736,0.359825
DemAge,-0.137767,1.000000,-0.172288,0.268434,0.207221,-0.302540
DemCluster,0.028571,-0.172288,1.000000,-0.108007,-0.054032,0.063521
PromSpend,-0.019621,0.268434,-0.108007,1.000000,0.033240,-0.079920
PromTime,-0.029736,0.207221,-0.054032,0.033240,1.000000,-0.053703
TargetBuy,0.359825,-0.302540,0.063521,-0.079920,-0.053703,1.000000


Проверка на наличие пустых значений:

In [9]:
new_data.isnull().any().any()

True

Так как имеем пустые значения, проведем анализ для поставленной задачи, по двум ситуациям:
    1. Удаление Nan (dropna())
    2. Замена Nan усредненными значениями (fillna())

## 1. Удаление Nan (dropna())

Подготовим данные:

In [10]:
data1 = new_data.dropna()

In [11]:
data1.isnull().any().any()

False

Выбираем количественные признаки и столбцы:

In [12]:
x = data1[['DemAffl',  'DemAge', 'DemCluster', 'PromSpend', 'PromTime']]
y = data1.values[::,10]

print(x)
print(y)

       DemAffl  DemAge  DemCluster  PromSpend  PromTime
0         10.0    76.0        16.0   16000.00       4.0
1          4.0    49.0        35.0    6000.00       5.0
2          5.0    70.0        27.0       0.02       8.0
3         10.0    65.0        51.0       0.01       7.0
4         11.0    68.0         4.0       0.01       8.0
...        ...     ...         ...        ...       ...
22216     13.0    49.0         8.0     500.00       9.0
22218     13.0    65.0         9.0    1500.00       5.0
22219     15.0    73.0        34.0    6053.06      12.0
22220      9.0    70.0        15.0    6000.00       5.0
22221     11.0    66.0         8.0    5000.00       5.0

[16408 rows x 5 columns]
[0 0 1 ... 0 0 0]


Обучаем дерево:

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.3)

y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [14]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(max_depth = 5, criterion = 'entropy')
tree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

Визуализируем модель:

In [15]:
from sklearn.tree import export_graphviz

import subprocess

export_graphviz(tree, out_file='/Users/giniatullinaalsuzamilevna/Desktop/Data/DATA-ANALYSYS_2-master/DZ/HW3/Organics_tree.dot', 
               rounded = True, proportion = False, 
                precision = 2, filled = True)

import pydot 
(graph, ) = pydot.graph_from_dot_file('/Users/giniatullinaalsuzamilevna/Desktop/Data/DATA-ANALYSYS_2-master/DZ/HW3/Organics_tree.dot')

graph.write_png('/Users/giniatullinaalsuzamilevna/Desktop/Data/DATA-ANALYSYS_2-master/DZ/HW3/Organics_tree.png')

<img src="Organics_tree.png">

In [16]:
from sklearn.metrics import classification_report, accuracy_score

clf = tree.fit(X_train, y_train)
y_pred = clf.predict(X_test)
result1 = classification_report(y_test, y_pred)
print("Отчет о классификации :",)
print (result1)
result2 = accuracy_score(y_test,y_pred)
print("Accuracy:",result2)

Отчет о классификации :
              precision    recall  f1-score   support

           0       0.82      0.93      0.87      3609
           1       0.69      0.44      0.54      1314

    accuracy                           0.80      4923
   macro avg       0.76      0.68      0.70      4923
weighted avg       0.79      0.80      0.78      4923

Accuracy: 0.7980905951655495


Проверяем точность на обучающей и тестовой выборке: 

In [17]:
tree.score(X_train, y_train) 

0.8019155420113191

In [18]:
tree.score(X_test,y_test)

0.7980905951655495

Посмотрим как параметры **max_depth** и **min_samples_leaf** влияют на построение дерева:

При **max_depth = 10**, **min_samples_leaf = 1**

In [19]:
tree_1 = DecisionTreeClassifier(max_depth = 10, min_samples_leaf = 1, criterion = 'entropy')
tree_1.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=10,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [20]:
tree_1.score(X_train, y_train) 

0.8295167609925991

In [21]:
tree_1.score(X_test, y_test) 

0.780418444038188

При **max_depth = 10**, **min_samples_leaf = 5**

In [22]:
tree_1_1 = DecisionTreeClassifier(max_depth = 10, min_samples_leaf = 5, criterion = 'entropy')
tree_1_1.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=10,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [23]:
tree_1_1.score(X_train, y_train) 

0.8265563778841968

In [24]:
tree_1_1.score(X_test, y_test) 

0.7859029047328865

При **max_depth = 15**, **min_samples_leaf = 5**

In [25]:
tree_2 = DecisionTreeClassifier(max_depth = 15, min_samples_leaf = 5, criterion = 'entropy')
tree_2.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=15,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [26]:
tree_2.score(X_train, y_train) 

0.8582498911623857

In [27]:
tree_2.score(X_test, y_test) 

0.7574649603900061

Влияние **max_depth** на дерево:

In [33]:
max_depth_values = range(1, 11)
scores_data = pd.DataFrame()

for max_depth in max_depth_values:    
    tree = DecisionTreeClassifier(criterion='entropy', max_depth=max_depth)
    tree.fit(X_train, y_train)
    train_score = tree.score(X_train, y_train)
    test_score = tree.score(X_test,y_test)
    
    temp_score_data = pd.DataFrame({'max_depth': [max_depth], 
                                    'train_score': [train_score],
                                    'test_score': test_score})
    scores_data = scores_data.append(temp_score_data)
    
scores_data

,max_depth,train_score,test_score
0,1,0.761167,0.756246
0,2,0.778755,0.782043
0,3,0.792773,0.790575
0,4,0.801393,0.800528
0,5,0.801916,0.798091
0,6,0.803309,0.797684
0,7,0.807227,0.792200
0,8,0.812103,0.793012
0,9,0.819068,0.792606
0,10,0.829952,0.779809


# 2. Замена Nan усредненными значениями (fillna())

Проведем анализ заменив Nan на усредненое значение:

In [45]:
data2 = new_data.fillna(new_data.median(axis=0), axis=0)

In [46]:
x = data2[['DemAffl',  'DemAge', 'DemCluster', 'PromSpend', 'PromTime']]
y = data2.values[::,10]

print(x)
print(y)

       DemAffl  DemAge  DemCluster  PromSpend  PromTime
0         10.0    76.0        16.0   16000.00       4.0
1          4.0    49.0        35.0    6000.00       5.0
2          5.0    70.0        27.0       0.02       8.0
3         10.0    65.0        51.0       0.01       7.0
4         11.0    68.0         4.0       0.01       8.0
...        ...     ...         ...        ...       ...
22218     13.0    65.0         9.0    1500.00       5.0
22219     15.0    73.0        34.0    6053.06      12.0
22220      9.0    70.0        15.0    6000.00       5.0
22221     11.0    66.0         8.0    5000.00       5.0
22222      3.0    75.0        53.0    6000.00       2.0

[22223 rows x 5 columns]
[0 0 1 ... 0 0 0]


In [53]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.3)

y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [54]:
from sklearn.tree import DecisionTreeClassifier

tree2 = DecisionTreeClassifier(max_depth = 5, criterion = 'entropy')
tree2.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [55]:
tree2.score(X_train, y_train) 

0.8077269220879404

In [56]:
tree2.score(X_test, y_test) 

0.7939103044847757

In [57]:
clf = tree2.fit(X_train, y_train)
y_pred2 = clf.predict(X_test)
result1 = classification_report(y_test, y_pred2)
print("Отчет о классификации :",)
print (result1)
result2 = accuracy_score(y_test,y_pred2)
print("Accuracy:",result2)

Отчет о классификации :
              precision    recall  f1-score   support

           0       0.81      0.94      0.87      4948
           1       0.68      0.38      0.49      1719

    accuracy                           0.79      6667
   macro avg       0.75      0.66      0.68      6667
weighted avg       0.78      0.79      0.77      6667

Accuracy: 0.7939103044847757


# Бинарное кодирование

In [5]:
from sklearn.preprocessing import LabelEncoder

categorical_columns = [c for c in data.columns if data[c].dtype.name == 'object']
print(categorical_columns)

def code_myohe(data, feature):
    for i in data[feature].unique():
        data[feature + "=" + i] = (data[feature] == i).astype(int)
        
le = LabelEncoder()
for c in categorical_columns:
    data[c] = data[c].astype(str)
    le.fit(data[c])
    data[data[c].name +'N'] = le.fit_transform(data[c])
    code_myohe(data, c)
    
data

['DemClusterGroup', 'DemGender', 'DemReg', 'DemTVReg', 'PromClass']


,ID,DemAffl,DemAge,DemCluster,DemClusterGroup,DemGender,DemReg,DemTVReg,PromClass,PromSpend,...,DemTVReg=Ulster,DemTVReg=S West,DemTVReg=Yorkshire,DemTVReg=Border,DemTVReg=N Scot,PromClassN,PromClass=Gold,PromClass=Silver,PromClass=Tin,PromClass=Platinum
0,140,10.0,76.0,16.0,C,U,Midlands,Wales & West,Gold,16000.00,...,0,0,0,0,0,0,1,0,0,0
1,620,4.0,49.0,35.0,D,U,Midlands,Wales & West,Gold,6000.00,...,0,0,0,0,0,0,1,0,0,0
2,868,5.0,70.0,27.0,D,F,Midlands,Wales & West,Silver,0.02,...,0,0,0,0,0,2,0,1,0,0
3,1120,10.0,65.0,51.0,F,M,Midlands,Midlands,Tin,0.01,...,0,0,0,0,0,3,0,0,1,0
4,2313,11.0,68.0,4.0,A,F,Midlands,Midlands,Tin,0.01,...,0,0,0,0,0,3,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22218,52834058,13.0,65.0,9.0,B,F,South East,London,Silver,1500.00,...,0,0,0,0,0,2,0,1,0,0
22219,52834376,15.0,73.0,34.0,D,U,South East,S & S East,Gold,6053.06,...,0,0,0,0,0,0,1,0,0,0
22220,52837057,9.0,70.0,15.0,B,F,North,Yorkshire,Gold,6000.00,...,0,0,1,0,0,0,1,0,0,0
22221,52838096,11.0,66.0,8.0,B,F,North,N West,Silver,5000.00,...,0,0,0,0,0,2,0,1,0,0


# Вопросы

**1.** Что такое решающее дерево? Как по построенному дереву найти прогноз для объекта?

Дерево решений - это некий бинарный классификатор. Правила классификации дерева дают прогноз каждому объекту дерева.

**2.** Зачем в вершинах нужны предикаты? Какие типы предикатов вы знаете? 

Предикаты - это и есть правила дерева которые делят информацию, они дают классификацию объектам. 
Предикаты бывают одномерными(сравнивают значение одного из признаков с константой) и многомерными(линейные, метрические).

**3.** Почему для любой выборки можно построить решающее дерево, имеющее нулевую ошибку на ней?

Для любой выборки можно построить решающее дерево, не допускающее на ней ни одной ошибки(даже с простыми одномерными предикатами). 
Можно сформировать дерево, в каждом листе которого находится ровно по одному объекту выборки.

**4.** Почему не рекомендуется строить небинарные деревья (т.е. имеющие больше двух потомков у каждой вершины)?

Потому что оценка объектов по одному предикату не даст полного анализа данных.

**5.** Как устроен жадный алгоритм построения дерева? Какие у него параметры?

Жадный алгоритм на каждом шагу делает локально наилучший выбор в надежде, что итоговое решение будет оптимальным. 

**6.** Зачем нужны критерии информативности?

Некий функционал качества. Оценивает качество распределения.